# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/Users/alirafieh/Downloads/Data Engineering/Projects/2. NoSQL- Cassandra/CassandaraProject


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# the code below can be uncomment for get total number of rows 
print(len(full_data_rows_list))
# the code below can be uncomment for check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding

#### After creating finilized CSV file titled <font color=red>event_datafile_new.csv</font>, located within current directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance  local machine (127.0.0.1)

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()

except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

#### Data Modeling and Creating Queries based on Following Questions:
1. Bring back the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

2. Bring back only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    
3. Bring back every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [8]:
# check columns name of CSV event file
sparkify_columns = pd.read_csv(os.getcwd() + '/event_datafile_new.csv')
sparkify_columns.columns

Index(['artist', 'firstName', 'gender', 'itemInSession', 'lastName', 'length',
       'level', 'location', 'sessionId', 'song', 'userId'],
      dtype='object')

#### 1. Creating query for question 1

In [12]:
## Query 1:  Give back the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

create1 = "CREATE TABLE IF NOT EXISTS song_artist_session "
create1 = create1 + """(session_id int,
                    session_item_number int,
                    artist_name text,
                    song_title text,
                    song_length float,
                    user_id int,
                    PRIMARY KEY ((session_id, session_item_number), user_id))"""

try:
    session.execute(create1)
except Exception as e:
    print(e)

query1 = "SELECT artist_name, song_title, song_length FROM song_artist_session WHERE session_id = %s AND \
            session_item_number =  %s"

In [13]:
# transfer data from csv file to table in Apache Cassandra
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

## Assign the INSERT statements into the `query` variable
        query = """INSERT INTO song_artist_session (user_id, session_id, session_item_number, 
                                                artist_name, song_title, song_length) """
        query = query + "VALUES  (%s, %s, %s, %s, %s, %s)"
        
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### SELECT Statement to verify that the data have been inserted into each table

In [14]:
## Add in the SELECT statement to verify the data was entered into the table
try:
    check1 = session.execute(query1, (338, 4))
except Exception as e:
    print(e)
    
for row in check1:
    print (row)

Row(artist_name='Faithless', song_title='Music Matters (Mark Knight Dub)', song_length=495.30731201171875)


#### 2. Creating query for question 2

In [15]:
## Query 2: Give back only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
create2 =  "CREATE TABLE IF NOT EXISTS song_playlist_session" 
create2 = create2 + """(user_id int,
                        session_id int,
                        session_item_number int,
                        artist_name text,
                        song_title text,
                        user_first_name text,
                        user_last_name text,
                        PRIMARY KEY ((user_id, session_id), session_item_number))"""


try:
    session.execute(create2)
except Exception as e:
    print(e)
    
    
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        query = """INSERT INTO song_playlist_session (user_id, session_id, session_item_number, artist_name, song_title,
        user_first_name, user_last_name ) """
        query = query + " VALUES  (%s, %s, %s, %s, %s, %s, %s)"
        
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4])) 

query2 = "SELECT artist_name, song_title, user_first_name, user_last_name FROM song_playlist_session\
          WHERE user_id = %s AND session_id = %s"

try:
    check2 = session.execute(query2, (10, 182))
except Expectation as e:
    print(e)
    
for row in check2:
    print (row)            

Row(artist_name='Down To The Bone', song_title="Keep On Keepin' On", user_first_name='Sylvie', user_last_name='Cruz')
Row(artist_name='Three Drives', song_title='Greece 2000', user_first_name='Sylvie', user_last_name='Cruz')
Row(artist_name='Sebastien Tellier', song_title='Kilometer', user_first_name='Sylvie', user_last_name='Cruz')
Row(artist_name='Lonnie Gordon', song_title='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', user_first_name='Sylvie', user_last_name='Cruz')


#### 3. Creating query for question 3

In [16]:
## Query 3: Give backevery user name (first and last) in my music app history who listened to the song\
##'All Hands Against His Own'
create3 =  "CREATE TABLE IF NOT EXISTS user_name_song"
create3 = create3 + """(song_title text, 
                       user_id int,
                       user_first_name text,
                       user_last_name text,
                       PRIMARY KEY ((song_title), user_id))"""

try:
    session.execute(create3)
except Exception as e:
    print(e)

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        query = """INSERT INTO user_name_song (song_title, user_id, user_first_name, user_last_name) """
        query = query + " VALUES  (%s, %s, %s, %s)"

        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

query3 = "SELECT user_first_name, user_last_name FROM user_name_song WHERE song_title = %s"

try:
    check3 = session.execute(query3, ('All Hands Against His Own', ))
except Exception as e:
    print(e)
    
    
for row in check3:
    print (row)

Row(user_first_name='Jacqueline', user_last_name='Lynch')
Row(user_first_name='Tegan', user_last_name='Levine')
Row(user_first_name='Sara', user_last_name='Johnson')


### Drop the tables before closing out the sessions

In [17]:
## Drop the table before closing out the sessions
query = "drop table song_artist_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

      
query = "drop table song_playlist_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
    
query = "drop table user_name_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()